In [1]:
import pandas as pd

In [2]:
# Standardization formula: z = (x - mean) / standard_deviation

class Standardiser():
    def __init__(self) -> None:
        pass
    
    def fit(self, X):
        self.size = X.shape[1]
        
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0)
        return self     #returns self, which allows method chaining
    
    def transform(self,X):
        if X.shape[1] != self.size:
            raise Exception("Wrong array dimensions!")
        return (X-self.mean)/self.std
        

In [4]:
#Read the data

dile_name = 'data-reg.csv'
raw_data = pd.read_csv(dile_name, header=None).to_numpy()

X = raw_data[:,:5]
Y = raw_data[:,-1]

print(f"Mean of all predictors{X.mean(axis=0)}")

scaler = Standardiser().fit(X)

X_scaled = scaler.transform(X)
print(f"Mean of all predictors standardised{X_scaled.mean(axis=0)}")





   
   


Mean of all predictors[0.20131508 8.97294864 0.13104092 0.11286003 0.22287212]
Mean of all predictors standardised[-2.53208760e-17  0.00000000e+00 -7.79103877e-18  4.28507132e-17
 -1.81790905e-17]
